Data: https://www.kaggle.com/nasa/meteorite-landings

In [23]:
import numpy as np
from PIL import Image
import pandas as pd
import folium
from IPython.display import display
import matplotlib.pyplot as plt
from PIL import Image
import webbrowser
import os
import plotly.express as px
import plotly.express as px
import plotly.graph_objects as go

Code to filter the meteorite data. Only needs to be run once.

In [24]:
meteorites = pd.read_csv("./data/meteorite-landings.csv", delimiter=",")
meteorites = meteorites.rename(columns={'name': 'Name', 'id': 'ID', 'nametype': 'Type', 'recclass': 'Class', 'mass': 'Mass', 'fall': 'Fell', 'year': 'Year', 'reclat': 'Latitude', 'reclong': 'Longitude'})

filtered_coords = (meteorites["Longitude"] >= -180) & (meteorites["Longitude"] <= 180) & ((meteorites["Longitude"] != 0) | (meteorites["Latitude"] != 0))
filtered_years = (meteorites["Year"] >= 860) & (meteorites["Year"] <= 2016)
filtered_mass = ~pd.isna(meteorites["Mass"])
filtered_type = ((meteorites["Type"] == "Valid"))
filtered_meteorites = meteorites[filtered_coords & filtered_years & filtered_type & filtered_mass]
filtered_meteorites.drop(["GeoLocation", "Type"], axis=1, inplace=True)
filtered_meteorites["Mass"] = filtered_meteorites["Mass"].div(1000) # convert gram to kilogram
filtered_meteorites.to_csv("./data/cleansed-data.csv")
filtered_meteorites = filtered_meteorites.sort_values(by="Year", ascending=True)
filtered_meteorites

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Name,ID,Class,Mass,Fell,Year,Latitude,Longitude
703,Nogata,16988,L6,0.4720,Fell,860.0,33.72500,130.75000
278,Elbogen,7823,"Iron, IID",107.0000,Fell,1399.0,50.18333,12.73333
852,Rivolta de Bassi,22614,Stone-uncl,0.1033,Fell,1490.0,45.48333,9.51667
283,Ensisheim,10039,LL6,127.0000,Fell,1491.0,47.86667,7.35000
5329,Campo del Cielo,5247,"Iron, IAB-MG",50000.0000,Found,1575.0,-27.46667,-60.58333
...,...,...,...,...,...,...,...,...
3403,Antelope,57455,H4,0.7540,Found,2012.0,40.90183,-118.54465
21698,Los Vientos 028,57338,H~5,12.1100,Found,2012.0,-24.68333,-69.76667
17386,Ksar Ghilane 010,55605,L5,0.0501,Found,2012.0,32.80600,9.83300
17252,Johannesburg,55765,H4,0.0630,Found,2012.0,35.37788,-117.63728


Code to plot the data

In [25]:
# fig = px.scatter_geo(filtered_meteorites,lat='Latitude',lon='Longitude',hover_name="Name",color="Mass",projection="natural earth",animation_frame="Year",animation_group="Mass")
# fig.update_layout(title = 'Meteorite Landings Across the World', title_x=.5)
# fig.write_html("images/animation.html")
# fig.show()

In [26]:
fig = px.scatter_geo(filtered_meteorites,lat='Latitude',color=np.log10(filtered_meteorites["Mass"]),size=(filtered_meteorites["Mass"] + 10000),range_color=[0.9, 4.1],color_continuous_scale="portland",lon='Longitude',hover_name="Name",projection="natural earth",animation_frame="Year")
fig.update_layout(title = 'Meteorite Landings Across the World', title_x=.5,margin_autoexpand=False,margin_b=200,margin_r=200)
fig.update_layout(coloraxis_colorbar=dict(
    title="Mass (kg)",
    tickvals=[1,2,3,4],
    ticktext=["10","100","1K","10K"],
))
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 100
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 200
fig.write_html("images/animation.html")